# Train and Deploy Your First Machine Learning Model on Amazon SageMaker

## Create SageMaker session

A SageMaker session needs to be initialized in order to start interacting the SageMaker service.

In [1]:
import boto3
import re

import os
import numpy as np
import pandas as pd
import sagemaker as sage

boto_session = boto3.Session(profile_name="up-sagemaker")
session = sage.Session(boto_session=boto_session)

## Upload the iris data for training 

Using the SageMaker session, which was initialized in the previous cell, the Iris data will be upload to S3. 

In [2]:
local_data_directory = 'data'
s3_prefix = 'up-sagemaker-iris'

data_location = session.upload_data(local_data_directory, key_prefix=s3_prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [3]:
from sagemaker import get_execution_role

account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/first-ml-model:latest'.format(account, region)

role = get_execution_role(session)
estimator = sage.estimator.Estimator(
                        image,
                        role, 
                        1, 
                        'ml.c4.2xlarge',
                        output_path="s3://{}/output".format(session.default_bucket()),
                        sagemaker_session=session
)

estimator.fit(data_location)

2020-03-20 22:11:40 Starting - Starting the training job...
2020-03-20 22:11:42 Starting - Launching requested ML instances...
2020-03-20 22:12:45 Starting - Preparing the instances for training......
2020-03-20 22:13:47 Downloading - Downloading input data
2020-03-20 22:13:47 Training - Downloading the training image...
2020-03-20 22:14:24 Uploading - Uploading generated training model
2020-03-20 22:14:24 Failed - Training job failed
Starting the training.
Traceback (most recent call last):
  File "/opt/program/train", line 102, in train
    labels = data_df.ix[:, 0]
  File "/usr/local/lib/python3.6/site-packages/pandas/core/generic.py", line 5274, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'ix'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/program/train", line 156, in <module>
    train()
  File "/opt/program/train", line 140, in train
    wit

UnexpectedStatusException: Error for Training job first-ml-model-2020-03-20-22-11-39-553: Failed. Reason: AlgorithmError: Exit Code: 1

## Deploy the model

Using the trained `Estimator`, a RESTful service will be initialized on Amazon SageMaker service.

In [4]:
from sagemaker.predictor import json_serializer

predictor = estimator.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

INFO:sagemaker:Creating model with name: first-ml-model-2018-09-16-17-21-57-001
INFO:sagemaker:Creating endpoint with name first-ml-model-2018-09-16-16-53-09-638


--------------------------------------------------!

## Sample some data and use it for a prediction

In [5]:
data_df = pd.read_csv("data/iris.csv", header=None, names=["feat_1", "feat_2", "feat_3", "feat_4"])

sampled_df = data_df.sample(10)

sampled_post_request_body = sampled_df.to_dict(orient='list')

sampled_post_request_body

{'feat_1': [5.6, 6.0, 4.8, 6.1, 5.1, 6.3, 5.1, 5.6, 5.9, 4.8],
 'feat_2': [2.5, 2.9, 3.4, 3.0, 3.8, 2.5, 3.8, 3.0, 3.2, 3.4],
 'feat_3': [3.9, 4.5, 1.6, 4.6, 1.9, 5.0, 1.5, 4.5, 4.8, 1.9],
 'feat_4': [1.1, 1.5, 0.2, 1.4, 0.4, 1.9, 0.3, 1.5, 1.8, 0.2]}

In [7]:
print(predictor.predict(sampled_post_request_body).decode('utf-8'))

{"predictions": ["versicolor", "versicolor", "setosa", "versicolor", "setosa", "virginica", "setosa", "versicolor", "versicolor", "setosa"]}


## Delete the endpoint

In [8]:
session.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: first-ml-model-2018-09-16-16-53-09-638
